In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings

# Filter out any warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message=".*Compiled the loaded model, but the compiled metrics have yet to be built.*")
warnings.filterwarnings("ignore", message=".*Error in loading the saved optimizer state.*")

# paths to my dataset folders
train_data_dir = '/Users/josephksaji/Desktop/Code/Python/Image Classification model - capstone/DATASET/TRAIN'
test_data_dir = '/Users/josephksaji/Desktop/Code/Python/Image Classification model - capstone/DATASET/TEST'

# image dimensions and batch size
img_height, img_width = 224, 224 
batch_size = 32

# data augmentation parameters for training set
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2  
)

# data augmentation parameters for testing set 
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Loading and Preprocessing training images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', 
    classes=['E', 'O', 'R']  
)

# Load and Preprocessing validation images
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', 
    classes=['E', 'O', 'R'] 
)

# Load and preprocess testing images
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,  
    classes=['E', 'O', 'R'] 
)

# the number of steps per epoch for training and validation
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size

# Accessing the loaded images and labels
batch_images, batch_labels = train_generator.__next__()
print("Shape of batch images:", batch_images.shape) 
print("Shape of batch labels:", batch_labels.shape) 

Found 25735 images belonging to 3 classes.
Found 6432 images belonging to 3 classes.
Found 3713 images belonging to 3 classes.
Shape of batch images: (32, 224, 224, 3)
Shape of batch labels: (32, 3)


In [2]:
tf.__version__

'2.16.1'

In [3]:
# CNN model 
import tensorflow as tf
from tensorflow.keras import layers, models

# Define custom attention layer
class AttentionLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Initialize weights for attention mechanism
        self.attention_weights = self.add_weight(shape=(input_shape[-1], 1),
                                                 initializer='random_normal',
                                                 trainable=True,
                                                 name='attention_weights')
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        # Compute attention scores for each spatial location
        attention_scores = tf.matmul(inputs, self.attention_weights)
        attention_scores = tf.squeeze(attention_scores, axis=-1)
        attention_weights = tf.nn.softmax(attention_scores, axis=-1)

        # Apply attention weights to input features
        weighted_features = inputs * tf.expand_dims(attention_weights, axis=-1)
        context_vector = tf.reduce_sum(weighted_features, axis=[1, 2])

        return context_vector

# Define CNN model with custom attention layer
def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        AttentionLayer(), 
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    return model

# Create an instance of the model
img_height, img_width = 224, 224
model = create_model((img_height, img_width, 3))


# Model summary
model.summary()

2024-03-27 11:50:19.103371: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-27 11:50:19.103406: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-27 11:50:19.103419: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-27 11:50:19.103490: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-27 11:50:19.103520: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ (None, 128)            │           128 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 439,107 (1.68 MB)

 Trainable params: 439,107 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Number of epochs
epochs = 100
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=None,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps
)
# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", test_accuracy)

Epoch 1/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 223s 274ms/step - accuracy: 0.8470 - loss: 0.3980 - val_accuracy: 0.7111 - val_loss: 1.1881
Epoch 2/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 187s 231ms/step - accuracy: 0.8581 - loss: 0.3694 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100


2024-03-27 12:10:28.909643: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-27 12:10:28.909680: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11232791536740727098
2024-03-27 12:10:28.909698: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-03-27 12:10:28.909703: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4738713395081679353
2024-03-27 12:10:28.909708: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10236901820959966261
2024-03-27 12:10:28.909712: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17946729879198359525
2024-03-

805/805 ━━━━━━━━━━━━━━━━━━━━ 234s 289ms/step - accuracy: 0.8685 - loss: 0.3460 - val_accuracy: 0.7830 - val_loss: 0.5849
Epoch 4/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 188s 232ms/step - accuracy: 0.8789 - loss: 0.3167 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100


2024-03-27 12:17:31.123131: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-27 12:17:31.123181: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11232791536740727098
2024-03-27 12:17:31.123207: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-03-27 12:17:31.123215: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4738713395081679353
2024-03-27 12:17:31.123220: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10236901820959966261
2024-03-27 12:17:31.123226: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17946729879198359525
2024-03-

805/805 ━━━━━━━━━━━━━━━━━━━━ 231s 285ms/step - accuracy: 0.8746 - loss: 0.3256 - val_accuracy: 0.7648 - val_loss: 0.8816
Epoch 6/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 190s 234ms/step - accuracy: 0.8850 - loss: 0.2992 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100


...

805/805 ━━━━━━━━━━━━━━━━━━━━ 216s 267ms/step - accuracy: 0.9521 - loss: 0.1261 - val_accuracy: 0.8613 - val_loss: 0.7173
Epoch 98/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 183s 225ms/step - accuracy: 0.9505 - loss: 0.1268 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 99/100


2024-03-27 17:46:34.772163: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-27 17:46:34.772206: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11232791536740727098
2024-03-27 17:46:34.772228: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-03-27 17:46:34.772234: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4738713395081679353
2024-03-27 17:46:34.772239: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10236901820959966261
2024-03-27 17:46:34.772244: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17946729879198359525
2024-03-

805/805 ━━━━━━━━━━━━━━━━━━━━ 224s 276ms/step - accuracy: 0.9515 - loss: 0.1239 - val_accuracy: 0.8640 - val_loss: 1.0319
Epoch 100/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 187s 230ms/step - accuracy: 0.9484 - loss: 0.1390 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


2024-03-27 17:53:25.638706: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-27 17:53:25.638752: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11232791536740727098
2024-03-27 17:53:25.638793: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-03-27 17:53:25.638801: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4738713395081679353
2024-03-27 17:53:25.638810: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10236901820959966261
2024-03-27 17:53:25.638815: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17946729879198359525
2024-03-

116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - accuracy: 0.9459 - loss: 0.3643
Test Accuracy: 0.907866358757019


In [7]:
# Evaluation of the model on the testing data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.9459 - loss: 0.3643
Test Loss: 0.37458211183547974
Test Accuracy: 0.907866358757019


In [9]:
# Saving the model 
model.save('/Users/josephksaji/Desktop/Code/Python/Image Classification model - capstone/modelCustom100.h5')

In [25]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import get_custom_objects

# Loading the trained model
get_custom_objects().update({'AttentionLayer': AttentionLayer})
model = load_model('/Users/josephksaji/Desktop/Code/Python/Image Classification model - capstone/modelCustom100.h5')

# Preprocessing the input image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224)) 
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.0 
    return img_array

# Predicting the waste image class
def predict_waste_image_class(image_path, model):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    waste_classes = ['Electronic', 'Biodegradable', 'Non-Biodegradable']
    predicted_waste_class = waste_classes[predicted_class_index]
    return predicted_waste_class

# Test with an example image
image_path = '/Users/josephksaji/Desktop/Code/Python/Image Classification model - capstone/prediction/duracell.jpeg'
predicted_class = predict_waste_image_class(image_path, model)
print('Predicted Waste Image Class:', predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Predicted Waste Image Class: Non-Biodegradable
